In [51]:
import pandas as pd

In [52]:
order_report=pd.read_excel("Company X - Order Report.xlsx")
pincode     =pd.read_excel("Company X - Pincode Zones.xlsx")
sku         =pd.read_excel("Company X - SKU Master.xlsx")
invoice     =pd.read_excel("Courier Company - Invoice.xlsx")
rate        =pd.read_excel("Courier Company - Rates.xlsx")

## Company X Order Report Processing

In [53]:
order_report.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


##### Check for Datatypes

In [54]:
order_report.dtypes

ExternOrderNo      int64
SKU               object
Order Qty        float64
dtype: object

##### Check for null values

In [55]:
order_report.isna().sum()

ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64

##### Check for Duplicates entire dataframe

In [56]:
print("Shape of 'Comapny X Order Report' Before Pre-Processing :",order_report.shape)
s=order_report.shape[0]
order_report.drop_duplicates(inplace=True)
t=order_report.shape[0]
print("Shape of 'Company X Order Report' After removing duplicates:",order_report.shape)
print(f'\nThere were {s-t} duplicates present in order report')

Shape of 'Comapny X Order Report' Before Pre-Processing : (400, 3)
Shape of 'Company X Order Report' After removing duplicates: (398, 3)

There were 2 duplicates present in order report


## Company X SKU(Stock keeping unit) Processing

In [57]:
sku.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


##### Check for Datatypes

In [58]:
sku.dtypes

SKU           object
Weight (g)     int64
dtype: object

##### Check for null values

In [59]:
sku.isna().sum()

SKU           0
Weight (g)    0
dtype: int64

##### Check for Duplicates entire dataframe

In [60]:
print("Shape of 'Comapny X SKU master' Before Pre-Processing :",sku.shape)
s=sku.shape[0]
sku.drop_duplicates(inplace=True)
t=sku.shape[0]
print("Shape of 'Company X SKU master' After removing duplicates:",sku.shape)
print(f'\nThere were {s-t} duplicates present in SKU')

Shape of 'Comapny X SKU master' Before Pre-Processing : (66, 2)
Shape of 'Company X SKU master' After removing duplicates: (65, 2)

There were 1 duplicates present in SKU


##### Check for Potential duplicates in Stock item(SKU)

In [61]:
duplicates = sku['SKU'].duplicated().sum()
if duplicates==0:
    print(f'There are {duplicates} duplicates in SKU ')
    print("All Items has unique SKU number")
else:
    print(f'There are {duplicates} duplicates in SKU ')
    print("If Same Item (SKU) has different weight, it results in ambiguity\n Sort out this issue")

There are 0 duplicates in SKU 
All Items has unique SKU number


## Calculating Total weight of Each Order

##### Merging 'Order Report' and 'sku' 

In [62]:
Order_Report_Weight=pd.merge(order_report, sku, on='SKU')
Order_Report_Weight.head()

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127


##### Calculating Total wight of each product in each order of Order Report 

In [63]:
Order_Report_Weight["Total weight as per X (KG)"]=Order_Report_Weight["Weight (g)"]*Order_Report_Weight["Order Qty"]
Order_Report_Weight.head()

,ExternOrderNo,SKU,Order Qty,Weight (g),Total weight as per X (KG)
0,2001827036,8904223818706,1.0,127,127.0
1,2001821995,8904223818706,1.0,127,127.0
2,2001819252,8904223818706,1.0,127,127.0
3,2001816996,8904223818706,1.0,127,127.0
4,2001814580,8904223818706,1.0,127,127.0


In [64]:
Order_Report_Weight.drop(columns=['Weight (g)','Order Qty'],inplace=True) 
#drop weight,Order Qty columns Since There is no use of these 2 columns any further calculations

In [65]:
Order_Report_Weight.shape

(398, 3)

In [66]:
Order_Report_Weight.head()

,ExternOrderNo,SKU,Total weight as per X (KG)
0,2001827036,8904223818706,127.0
1,2001821995,8904223818706,127.0
2,2001819252,8904223818706,127.0
3,2001816996,8904223818706,127.0
4,2001814580,8904223818706,127.0


##### Calculating Total Weight of Each order by grouping OrderNo

In [67]:
order_weight=Order_Report_Weight.groupby('ExternOrderNo').agg({'Total weight as per X (KG)': 'sum',}).reset_index()
order_weight.head()

,ExternOrderNo,Total weight as per X (KG)
0,2001806210,220.0
1,2001806226,480.0
2,2001806229,500.0
3,2001806232,1302.0
4,2001806233,245.0


In [68]:
order_weight.shape

(124, 2)

## Courier Company Invoice Processing

In [69]:
invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


##### Check for Datatypes

In [70]:
invoice.dtypes

AWB Code                  int64
Order ID                  int64
Charged Weight          float64
Warehouse Pincode         int64
Customer Pincode          int64
Zone                     object
Type of Shipment         object
Billing Amount (Rs.)    float64
dtype: object

##### Check for null values

In [71]:
invoice.isna().sum()

AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

##### check for duplicates

In [72]:
invoice.shape

(124, 8)

In [73]:
print("Shape of Courier Comapny invoice Before Pre-Processing :",invoice.shape)
s=invoice.shape[0]
invoice.drop_duplicates(inplace=True)
t=invoice.shape[0]
print("Shape of Couier Company invoice After removing duplicates:",invoice.shape)
print(f'\nThere were {s-t} duplicates present in invoice')

Shape of Courier Comapny invoice Before Pre-Processing : (124, 8)
Shape of Couier Company invoice After removing duplicates: (124, 8)

There were 0 duplicates present in invoice


##### Check for Potential duplicates in Order ID,AWB code

In [74]:
print(invoice['Order ID'].duplicated().sum())
print(invoice['AWB Code'].duplicated().sum())

0
0


## Create New Dataframe by merging Order weight and Invoice to 'cross check order Weight'

In [75]:
invoice_new=pd.merge(order_weight, invoice, left_on='ExternOrderNo', right_on='Order ID')
invoice_new.head()

,ExternOrderNo,Total weight as per X (KG),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,220.0,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,480.0,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,500.0,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1302.0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,245.0,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3


### Convering Weights from grams to KG

In [76]:
invoice_new["Total weight as per X (KG)"]=invoice_new["Total weight as per X (KG)"]/1000
invoice_new.head()

,ExternOrderNo,Total weight as per X (KG),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.220,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.480,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.500,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1.302,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,0.245,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3


### Calculating Weight in Weight slabs

In [77]:
def weight(a):            #Function to calculate weight slab
    b=a%1                #check for remaining grams of weight after getting complete full KGs(0,1,2,...)
    if 0.0<b<=0.5:   
        a//=1
        a+=0.5          #if remaining grams are less than or equal to 0.5g and more than 0.0g add only 0.5 weight slabs
    elif b>0.5:
        a//=1
        a+=1            #if remaining grams are more than 0.5g add only 1.0 weight slabs
    return a
    
invoice_new['Weight slab as per X (KG)']=invoice_new['Total weight as per X (KG)'].apply(lambda x:weight(x))
invoice_new['Weight slab charged by Courier Company (KG)']=invoice_new['Charged Weight'].apply(lambda x:weight(x))
invoice_new.head()

,ExternOrderNo,Total weight as per X (KG),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab as per X (KG),Weight slab charged by Courier Company (KG)
0,2001806210,0.220,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,0.5,3.0
1,2001806226,0.480,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,0.5,1.0
2,2001806229,0.500,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,0.5,1.0
3,2001806232,1.302,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5,1.5
4,2001806233,0.245,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,0.5,1.0


In [78]:
invoice_new.rename(columns={'Zone':'Delivery Zone charged by Courier Company'},inplace=True)

In [79]:
invoice_new['Warehouse Pincode'].unique()

array([121003], dtype=int64)

only one warehouse is present

In [80]:
invoice_new.drop(columns='Warehouse Pincode',inplace=True)

## Pincode Dataframe Processing

In [81]:
pincode.head()

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


##### Check for Datatypes

In [82]:
pincode.dtypes

Warehouse Pincode     int64
Customer Pincode      int64
Zone                 object
dtype: object

##### Check for null values

In [83]:
pincode.isna().sum()

Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

##### Check for Duplicates

In [84]:
print("Shape of pincode Before Pre-Processing :",pincode.shape)
s=pincode.shape[0]
pincode.drop_duplicates(inplace=True)
t=pincode.shape[0]
print("Shape of pincode After removing duplicates:",pincode.shape)
print(f'\nThere were {s-t} duplicates present in pincode')

Shape of pincode Before Pre-Processing : (124, 3)
Shape of pincode After removing duplicates: (108, 3)

There were 16 duplicates present in pincode


In [85]:
pincode['Warehouse Pincode'].unique()

array([121003], dtype=int64)

only one warehouse is present

In [86]:
pincode.drop(columns='Warehouse Pincode',inplace=True)

In [87]:
pincode.rename(columns={'Zone':'Delivery Zone as per X'},inplace=True)

## Create New DataFrame by merging invoice_new and pincode to 'cross check pincode'

In [88]:
Resultant=pd.merge(invoice_new, pincode, on='Customer Pincode')
Resultant.head()

,ExternOrderNo,Total weight as per X (KG),AWB Code,Order ID,Charged Weight,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Billing Amount (Rs.),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X
0,2001806210,0.220,1091117221940,2001806210,2.92,140604,b,Forward charges,174.5,0.5,3.0,b
1,2001806226,0.480,1091117222065,2001806226,0.68,723146,d,Forward charges,90.2,0.5,1.0,d
2,2001806229,0.500,1091117222080,2001806229,0.71,421204,d,Forward charges,90.2,0.5,1.0,d
3,2001806232,1.302,1091117222124,2001806232,1.30,507101,d,Forward charges,135.0,1.5,1.5,d
4,2001806233,0.245,1091117222135,2001806233,0.78,263139,b,Forward charges,61.3,0.5,1.0,b


In [89]:
rate.head()

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [90]:
Resultant['Type of Shipment'].unique()

array(['Forward charges', 'Forward and RTO charges'], dtype=object)

## Calculating Delivery Charges

In [91]:
def charge(h,z,a):                   #Function To calculate delivery charges
    price=0
    string1='fwd_'+z+'_fixed'        #For all orders There is fixed charges,we are calculating rto cost for first 0.5 weigh slab here.
    price+=rate[string1][0]
    b=a-0.5                      #calculate remaining weight
    if  b>=0.5:                  #if there is remaining weight ,it will be atleast 0.5 that's why we used this condition
        n=b//0.5                   #remaining weight will be in the multiple of 0.5
        string2='fwd_'+z+'_additional'
        price+=(n*rate[string2][0])  #if weight is more than 0.5 weigh slab, add additional forward cost here w.r.t the propotion. 
    if h=='Forward and RTO charges':   #if shipment contains RTO charges, we calculate rto cost for first 0.5 weigh slab.
        string3='rto_'+z+'_fixed'
        price+=rate[string3][0]
        b=a-0.5
        if b>=0.5:                       #if there is remaining weight ,it will be atleast 0.5.
            n=b//0.5                      #remaining weight will be in the multiple of 0.5
            string4='rto_'+z+'_additional'
            price+=(n*rate[string4][0])  #if weight is more than 0.5 weigh slab, add additional rto cost here w.r.t the propotion.
            
    return price

Resultant['Expected Charge as per X (Rs.)']=Resultant.apply(lambda x:charge(x['Type of Shipment'],x['Delivery Zone as per X'],x['Weight slab as per X (KG)']),axis=1)

In [92]:
Resultant.head()

,ExternOrderNo,Total weight as per X (KG),AWB Code,Order ID,Charged Weight,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Billing Amount (Rs.),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.)
0,2001806210,0.220,1091117221940,2001806210,2.92,140604,b,Forward charges,174.5,0.5,3.0,b,33.0
1,2001806226,0.480,1091117222065,2001806226,0.68,723146,d,Forward charges,90.2,0.5,1.0,d,45.4
2,2001806229,0.500,1091117222080,2001806229,0.71,421204,d,Forward charges,90.2,0.5,1.0,d,45.4
3,2001806232,1.302,1091117222124,2001806232,1.30,507101,d,Forward charges,135.0,1.5,1.5,d,135.0
4,2001806233,0.245,1091117222135,2001806233,0.78,263139,b,Forward charges,61.3,0.5,1.0,b,33.0


In [93]:
Resultant.rename(columns={'Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)'},inplace=True)

In [94]:
Resultant.head()

,ExternOrderNo,Total weight as per X (KG),AWB Code,Order ID,Charged Weight,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.)
0,2001806210,0.220,1091117221940,2001806210,2.92,140604,b,Forward charges,174.5,0.5,3.0,b,33.0
1,2001806226,0.480,1091117222065,2001806226,0.68,723146,d,Forward charges,90.2,0.5,1.0,d,45.4
2,2001806229,0.500,1091117222080,2001806229,0.71,421204,d,Forward charges,90.2,0.5,1.0,d,45.4
3,2001806232,1.302,1091117222124,2001806232,1.30,507101,d,Forward charges,135.0,1.5,1.5,d,135.0
4,2001806233,0.245,1091117222135,2001806233,0.78,263139,b,Forward charges,61.3,0.5,1.0,b,33.0


In [95]:
Resultant['Difference Between Expected Charges and Billed Charges (Rs.)']=Resultant['Expected Charge as per X (Rs.)']-Resultant['Charges Billed by Courier Company (Rs.)']

In [96]:
Resultant.head()

,ExternOrderNo,Total weight as per X (KG),AWB Code,Order ID,Charged Weight,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,0.220,1091117221940,2001806210,2.92,140604,b,Forward charges,174.5,0.5,3.0,b,33.0,-141.5
1,2001806226,0.480,1091117222065,2001806226,0.68,723146,d,Forward charges,90.2,0.5,1.0,d,45.4,-44.8
2,2001806229,0.500,1091117222080,2001806229,0.71,421204,d,Forward charges,90.2,0.5,1.0,d,45.4,-44.8
3,2001806232,1.302,1091117222124,2001806232,1.30,507101,d,Forward charges,135.0,1.5,1.5,d,135.0,0.0
4,2001806233,0.245,1091117222135,2001806233,0.78,263139,b,Forward charges,61.3,0.5,1.0,b,33.0,-28.3


#### Dropping Unneccessary columns,renaming some columns

In [97]:
Resultant.drop(columns=['ExternOrderNo','Customer Pincode'],inplace=True)
Resultant.rename(columns={'AWB Code':'AWB Number','Charged Weight':'Total weight as per Courier Company (KG)'},inplace=True)
Resultant.head()

,Total weight as per X (KG),AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,0.220,1091117221940,2001806210,2.92,b,Forward charges,174.5,0.5,3.0,b,33.0,-141.5
1,0.480,1091117222065,2001806226,0.68,d,Forward charges,90.2,0.5,1.0,d,45.4,-44.8
2,0.500,1091117222080,2001806229,0.71,d,Forward charges,90.2,0.5,1.0,d,45.4,-44.8
3,1.302,1091117222124,2001806232,1.30,d,Forward charges,135.0,1.5,1.5,d,135.0,0.0
4,0.245,1091117222135,2001806233,0.78,b,Forward charges,61.3,0.5,1.0,b,33.0,-28.3


#### Rearranging columns

In [98]:
Resultant=Resultant.iloc[:,[2,1,0,7,3,8,9,4,10,6,11]]
Resultant.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.0,61.3,-28.3


## Summary Table

In [99]:
count1=Resultant[Resultant['Difference Between Expected Charges and Billed Charges (Rs.)']==0].shape[0]
count2=Resultant[Resultant['Difference Between Expected Charges and Billed Charges (Rs.)']<0].shape[0]
count3=Resultant[Resultant['Difference Between Expected Charges and Billed Charges (Rs.)']>0].shape[0]

total_invoice_amount=Resultant.loc[Resultant['Difference Between Expected Charges and Billed Charges (Rs.)']==0,'Charges Billed by Courier Company (Rs.)'].sum()

total_overcharging_amount=Resultant.loc[Resultant['Difference Between Expected Charges and Billed Charges (Rs.)']<0,'Difference Between Expected Charges and Billed Charges (Rs.)'].sum()
total_undercharging_amount=Resultant.loc[Resultant['Difference Between Expected Charges and Billed Charges (Rs.)']>0,'Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

total_overcharging_amount=abs(total_overcharging_amount)
print(total_invoice_amount)
print(total_overcharging_amount)
print(total_undercharging_amount)

Count_data=[count1,count2,count3]
Amount_data=[total_invoice_amount,total_overcharging_amount,total_undercharging_amount]
labels=['Total orders where X has been correctly charged','Total Orders where X has been overcharged','Total Orders where X has been undercharged']

summary=pd.DataFrame(data={'Count':Count_data,'Amount (Rs.)':Amount_data},index=labels)
summary

1207.3
4483.199999999999
575.0999999999999


,Count,Amount (Rs.)
Total orders where X has been correctly charged,15,1207.3
Total Orders where X has been overcharged,85,4483.2
Total Orders where X has been undercharged,24,575.1


## Storing Results in Excel Workbooks

In [100]:
writer = pd.ExcelWriter('Final Results.xlsx', engine='xlsxwriter')
Resultant.to_excel(writer, sheet_name='Order level calculation',index=False,columns=Resultant.columns)
summary.to_excel(writer, sheet_name='Summary')

worksheet1 = writer.sheets['Order level calculation']
worksheet1.set_column('A:B', 20)          #changing width of certain columns of excel sheet
worksheet1.set_column('C:M', 40)

worksheet2 = writer.sheets['Summary']
worksheet2.set_column('A:A', 50)
worksheet2.set_column('B:C',13)


writer.save()